**Baseline classifier model. Op basis van TF-IDF Logistische regressie. Moet class imbalance nog aanpakken.**

In [2]:
""""
wat gebeurd is: eerst GridSearchCV gerund en daarna gezien dat deze configuratie met l1 dus de beste is"
daarna dus de l1 configuratie gebruikt en de beste parameters nog aan het zoeken"
"""


'"\nwat gebeurd is: eerst GridSearchCV gerund en daarna gezien dat deze configuratie met l1 dus de beste is"\ndaarna dus de l1 configuratie gebruikt en de beste parameters nog aan het zoeken"\n'

In [3]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd


import os

In [4]:
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

# 2. Data verkennen
print("Dataset overzicht:")
print(df.info())
print(df.head())

Dataset overzicht:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102015 entries, 0 to 102014
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   context      101405 non-null  object
 1   question     102014 non-null  object
 2   statistical  102015 non-null  int64 
 3   theme        102015 non-null  object
 4   file_name    102015 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.9+ MB
None
                                             context  \
0  Ondertussen is de eerstelijnszone BruZEL al me...   
1  Ondertussen is de eerstelijnszone BruZEL al me...   
2  Ondertussen is de eerstelijnszone BruZEL al me...   
3  Ondertussen is de eerstelijnszone BruZEL al me...   
4  Ondertussen is de eerstelijnszone BruZEL al me...   

                                            question  statistical  \
0  Zoals alle eerstelijnszones kreeg ook BruZEL h...            0   
1                            a)Wat liep er moei

In [4]:
# 3. Missende waarden aanpakken
df = df.dropna(subset=["question"])  # Verwijder lege vragen
df["context"].fillna("", inplace=True)  # Vul missende context met lege string

# 4. Stopwoordenlijst (Nederlands, handmatig)
dutch_stopwords = {"de", "het", "een", "en", "van", "ik", "te", "dat", "die", "in", "je", "is", "niet", "op", "aan", "met", "als", "voor", "zijn", "was", "heeft", "heb", "om", "bij", "of", "geen", "dan", "toch", "maar", "wel", "meer", "doen", "ook", "kan", "mijn", "zo", "dus", "zou", "kunnen"}

def clean_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'\n', ' ', text)  # Nieuwe regels verwijderen
    text = re.sub(r'\b[a-z]\)\s+', ' ', text)  # Verwijder enumeraties zoals "a) b) c)" en voeg spatie toe
    text = re.sub(r'\b\d+\.\b', '', text)  # Verwijder numerieke opsommingen zoals "1. 2. 3."
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Speciale tekens verwijderen
    text = " ".join([word for word in text.split() if word not in dutch_stopwords])  # Stopwoorden verwijderen
    text = re.sub(r'\s+', ' ', text).strip()  # Dubbele spaties verwijderen
    return text

# 5. Data opschonen
df["clean_question"] = df["question"].apply(clean_text)

C:\Users\jefva\AppData\Local\Temp\ipykernel_17592\254478967.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)  # Vul missende context met lege string


In [6]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["clean_question"])
y = df["theme"]

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from imblearn.over_sampling import RandomOverSampler

# ✅ Get the class distribution
theme_counts = y_train.value_counts()
median_count = theme_counts.median()  # Threshold for balancing

# ✅ Define oversampling strategy: Increase rare themes to the median count
sampling_strategy = {theme: int(median_count) for theme in theme_counts.index if theme_counts[theme] < median_count}

# ✅ Apply Random Oversampling
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

print(f"Original dataset size: {X_train.shape[0]}")
print(f"After Random Oversampling: {X_train_resampled.shape[0]}")


Original dataset size: 81564
After Random Oversampling: 95744


In [6]:
# ✅ Hyperparameter tuning met GridSearchCV
# Conditional grid - alleen 'elasticnet' krijgt l1_ratio
param_grid = {
    'C': [0.1, 1, 10],  # Minder waarden testen
    'penalty': ['l1', 'l2'],  # ElasticNet weglaten als dat niet veel verschil maakt
    'solver': ['saga']
}

grid = GridSearchCV(
    LogisticRegression(max_iter=500),  # Model
    param_grid,  # Zoekruimte
    cv=5,  # 5-fold cross-validation
    scoring="f1_weighted",  # Optimaliseer voor F1-score
    n_jobs=-1,  # Parallel berekenen
    verbose=2  # ✅ Geeft gedetailleerde logs per iteratie
)

print("GridSearchCV wordt uitgevoerd... Dit kan even duren 🚀")
grid.fit(X_train_resampled, y_train_resampled)  # GridSearch starten

# ✅ Print beste parameters en scores
print(f"Beste parameters: {grid.best_params_}")
best_model = grid.best_estimator_

# ✅ Evaluatie op de testset
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

# ✅ Compute Accuracy, Precision, Recall, and F1-score (same format as BERT output)
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted", zero_division=1)

# ✅ Format Output as DataFrame
baseline_results = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1"],
    "Score": [accuracy, precision, recall, f1]
})

# ✅ Display results in a table format
print(baseline_results)


GridSearchCV wordt uitgevoerd... Dit kan even duren 🚀
Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: 